In [1]:
import gc
import joblib
import polars as pl
from src.cluster_cust_data import encode_features, predict_cluster_with_centroids
from src.cust_feature_eng import extract_customer_features
from src.flt_feature_eng import extract_flight_features, prepare_combined_data
from src.ranking import train_segment_based_ranker
from src.ranking_eval import  test_lgb_ranker, test_segment_based_ranker

%load_ext autoreload
%autoreload 2

### Train Ranking

In [2]:
# Get Training data set
train_df = pl.read_parquet('data/train_combined_features.parquet')
train_df.head(100)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",1,0.0,0.04,1.0,25,3.258097,0,1,"""TLK""",0,0,0,1,1,2,160,155,29,160,155,315,15,6,16,6,9,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,5,1.0,0.0,1.80224,0
1,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",0,4.166667,0.08,3.028015,25,3.258097,1,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,5,1.0,0.0,3.60448,1
2,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",0,29.166667,0.32,3.18023,25,3.258097,0,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,5,1.0,0.0,14.417919,1
3,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",0,54.166667,0.56,4.849562,25,3.258097,1,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,5,1.0,0.0,25.231359,1
4,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",0,79.166667,0.8,5.097726,25,3.258097,0,1,"""TLK""",0,0,0,2,2,4,250,245,29,445,505,950,9,6,14,6,22,…,0.962264,0.0,0.0,1.981132,1,-0.253891,93.370333,68.147027,45.055998,0.141509,0.619049,1,1.018868,0.0,1.0,-32.58008,5.4944,0.052632,45.358188,26.5,18,1.433237,0.056604,0.028302,1.0,1,0.0,0.0,0.0,106,0.235849,18,5,1.0,0.0,36.044799,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
144,0,2000.0,0.0,1.0,0.0,0.0,0.0,1.0,1,"""e109b5

In [3]:
train_df.group_by('customer_segment').len().select(['customer_segment', 'len']).rows()

[(8, 66857),
 (3, 979087),
 (10, 4961),
 (4, 16477),
 (6, 9397),
 (5, 2717359),
 (7, 6429),
 (2, 3622),
 (11, 1646712),
 (0, 6249166),
 (9, 663189),
 (1, 282679)]

In [24]:
segment_stats = (
    train_df
    .group_by(['customer_segment', 'ranker_id'])
    .agg([
        pl.first('total_searches').alias('customer_total_searches'),
        pl.len().alias('flights_per_search')
    ])
    .group_by('customer_segment')
    .agg([
        pl.len().alias('total_searches'),                    # Count of search sessions
        pl.col('customer_total_searches').n_unique().alias('num_customers'),  # Since each unique total_searches value represents a customer
        pl.col('flights_per_search').min().alias('min_flights_per_search'),
        pl.col('flights_per_search').max().alias('max_flights_per_search'),
        pl.col('flights_per_search').mean().alias('avg_flights_per_search')
    ])
    .sort('customer_segment')
)
segment_stats

customer_segment,total_searches,num_customers,min_flights_per_search,max_flights_per_search,avg_flights_per_search
i64,u32,u32,u32,u32,f64
0,43116,1947,2,6063,144.938445
1,2409,397,1,2773,117.342881
2,9,4,95,1082,402.444444
3,7797,888,2,7841,125.572271
4,74,38,3,639,222.662162
…,…,…,…,…,…
7,4,3,1047,2237,1607.25
8,1816,204,1,1126,36.815529
9,4525,663,2,7678,146.561105


In [4]:
train_df.shape

(12645935, 121)

In [4]:
ranker_models = train_segment_based_ranker(train_df)

2025-09-03 16:13:07.069751: Training segment 0 with 6249166 rows
2025-09-03 16:19:46.029505: Training segment 1 with 282679 rows
2025-09-03 16:20:32.486334: Training segment 2 with 3622 rows
2025-09-03 16:20:33.339276: Training segment 3 with 979087 rows
2025-09-03 16:23:00.127873: Training segment 4 with 16477 rows
2025-09-03 16:23:02.597016: Training segment 5 with 2717359 rows
2025-09-03 16:27:09.828914: Training segment 6 with 9397 rows
2025-09-03 16:27:12.031549: Training segment 7 with 6429 rows
2025-09-03 16:27:13.164240: Training segment 8 with 66857 rows
2025-09-03 16:27:23.143047: Training segment 9 with 663189 rows
2025-09-03 16:28:47.806004: Training segment 10 with 4961 rows
2025-09-03 16:28:48.803125: Training segment 11 with 1646712 rows


In [5]:
joblib.dump(ranker_models, 'data/segment_ranker_models.joblib')
ranker_models

{0: <lightgbm.basic.Booster at 0x139b8026d80>,
 1: <lightgbm.basic.Booster at 0x139b6fdc920>,
 2: <lightgbm.basic.Booster at 0x139b80a8a70>,
 3: <lightgbm.basic.Booster at 0x139b8408e60>,
 4: <lightgbm.basic.Booster at 0x139b800b0e0>,
 5: <lightgbm.basic.Booster at 0x139b80a8260>,
 6: <lightgbm.basic.Booster at 0x139b80108f0>,
 7: <lightgbm.basic.Booster at 0x139b6fc4290>,
 8: <lightgbm.basic.Booster at 0x139b8404ef0>,
 9: <lightgbm.basic.Booster at 0x139b83e4500>,
 10: <lightgbm.basic.Booster at 0x139b6f9b350>,
 11: <lightgbm.basic.Booster at 0x139b8009d00>,
 99: <lightgbm.basic.Booster at 0x139b843c6e0>}

In [5]:
# Remove the combined features dataframe to free up memory
del train_df
gc.collect()

76

### Test Ranking

In [5]:
# Get Test data set
test_df = pl.read_parquet('data/test_df.parquet')
test_df.head()

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
70,true,62836,81,null,36,true,false,"""2024-05-20T15:40:00""","""2024-05-20T13:20:00""","""01:20:00""","""319""","""UUS""","""UUS""",0.0,0.0,1.0,"""KHV""","""01:20:00""","""5628""","""SU""","""HZ""",2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4600.0,null,1.0,0.0,null,0.0,0.0,1,3382768,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,"""KHVUUS""",true,946.0,7246.0,0,70
71,true,62836,81,null,36,true,false,"""2024-05-20T15:40:00""","""2024-05-20T13:20:00""","""01:20:00""","""319""","""UUS""","""UUS""",1.0,0.0,1.0,"""KHV""","""01:20:00""","""5628""","""SU""","""HZ""",2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4600.0,null,1.0,4600.0,null,1.0,0.0,1,3382768,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,"""KHVUUS""",true,946.0,9126.0,1,71
72,true,62836,81,null,36,true,false,"""2024-05-20T15:40:00""","""2024-05-20T13:20:00""","""01:20:00""","""319""","""UUS""","""UUS""",2.0,0.0,1.0,"""KHV""","""01:20:00""","""5628""","""SU""","""HZ""",2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,1.0,0.0,null,1.0,0.0,1,3382768,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,"""KHVUUS""",true,946.0,13636.0,0,72
73,true,62836,81,null,36,true,false,"""2024-05-20T15:40:0

In [6]:
# Customer Feature Engineering
cust_data = extract_customer_features(test_df)
print(f'Generated {len(cust_data.columns)} customer features for {len(cust_data)} customers')

Generated 58 customer features for 13781 customers


In [7]:
# Get the encoders, scaler, and reducer from the joblib file for the Train data clustering process
data_encoders_scaler_reducer = joblib.load('data/data_encoders_scaler_reducer.joblib')
print('loaded encoders, sclaer, reducer from joblib file')

encoders = data_encoders_scaler_reducer['encoders']
scaler = data_encoders_scaler_reducer['scaler']
reducer = data_encoders_scaler_reducer['reducer']


loaded encoders, sclaer, reducer from joblib file


In [8]:
# Encode customer features
cust_data_encoded, _ = encode_features(cust_data, encoders)

In [9]:
# Scale customer features
scaled_features = scaler.transform(cust_data_encoded.to_pandas())

In [10]:
# Dimensionality reduction
reduced_features = reducer.transform(scaled_features)

In [11]:
# Load Agglomeratvie clustering results from Train to get model centroids
clustering_results = joblib.load('data/agglomerative_model.joblib')
print(f'Loaded Agglomerative Clustering results from joblib file for {clustering_results['n_clusters']} clusters')

Loaded Agglomerative Clustering results from joblib file for 12 clusters


In [12]:
# Generate cluster label predictions for Test data
cluster_segments = predict_cluster_with_centroids(reduced_features, clustering_results['centroids'])

In [13]:
# Add predicted cluster labels to the customer data
cust_data = cust_data.with_columns(pl.Series('customer_segment', cluster_segments[0]))
cust_data.head(1000)

profileId,company_id,sex,nationality,frequent_flyer,is_vip,by_self,has_corp_codes,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,most_common_departure_airport,unique_departure_airports,most_common_carrier,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class,weekday_preference,weekend_travel_rate,time_of_day_variance,night_flight_preference,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment
i64,i64,i8,i64,str,i8,i8,i8,u32,f64,u32,i32,i32,f64,f64,str,u32,str,u32,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64
734059,27334,1,36,"""S7/SU""",0,1,1,44,0.0,1,93,94,93.363636,93.0,"""OVB""",1,"""SU""",6,1.0,2.0,1.454545,1,0.0,6.071108,0.045455,0.977273,0.113636,0.0,2.0,1,0.023806,109.128758,104.265943,47.410148,0.454545,0.59835,1,1.186047,40.0,1.0,-50.592047,5.258985,0.5,52.795983,44.0,1,1.00391,0.136364,0.022727,1.0,1,0.0,0.0,0.0,44,0.0,1,9
3088521,60187,1,36,"""SU/N4/S7""",0,1,1,17,0.0,1,1,2,1.823529,2.0,"""KUF""",1,"""SU""",2,1.0,1.0,1.0,3,0.0,5.036631,0.0,0.941176,0.0,0.0,1.882353,2,0.287031,55.382486,32.776838,50.0,0.0,0.608088,1,0.941176,0.0,1.0,-38.93115,5.0,0.5,45.0,17.0,1,0.911765,0.117647,0.058824,0.0,1,0.0,0.0,0.0,17,0.0,1,5
3568400,61145,1,36,"""""",0,1,1,175,1.0,1,4,5,4.274286,4.0,"""SVX""",1,"""SU""",5,1.0,2.0,1.045714,3,0.0,5.480373,0.08,0.994286,0.0,0.016349,1.291429,1,0.596079,79.419596,36.26016,49.885057,0.045714,0.62543,1,0.862857,0.0,0.0,-41.823332,-4.988506,0.5,45.856814,175.0,1,1.068571,0.028571,0.005714,1.0,1,0.0,0.0,0.0,175,1.0,1,0
3485953,57323,1,36,"""""",0,1,1,60,0.0,3,1,7,4.933333,6.0,"""UFA""",3,"""SU""",4,1.0,2.0,1.25,4,0.0,5.205251,0.233333,0.95,0.0,0.0,1.916667,1,-0.082569,133.614961,82.251888,48.395833,0.25,0.624553,1,1.283333,0.0,0.0,-42.465352,-4.839583,0.142857,49.197917,20.0,6,0.822222,0.066667,0.05,1.0,1,0.0,0.0,0.0,60,0.0,6,0
2005655,60691,1,36,"""SU""",0,1,0,24,1.0,1,3,3,3.0,3.0,"""LED""",1,"""SU""",1,1.0,1.0,1.0,7,1.0,0.0,0.0,0.958333,0.0,0.65,1.0,1,0.0,91.401852,32.809371,48.550725,0.0,0.598714,1,0.75,0.0,1.0,0.0,5.144928,1.0,44.275362,24.0,0,1.0,0.041667,0.041667,0.0,1,0.0,0.0,0.0,0,0.0,0,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3416225,52573,1,36,"""""",0,1,1,46,0.0,2,0,2,0.934783,1.0,"""SGC""",2,"""SU""",4,1.0,1.0,1.0,7,0.73913,6.21732,0.347826,0.956522,0.0,0.0,1.869565,2,0.327398,96.075492,55.326733,47.96725,0.0,0.623489,2,0.891304,0.0,0.0,-50.536212,-4.796725,0.333333,43.983625,23.0,2,0.934783,0.086957,0.043478,0.0,1,0.0,0.0,0.0,46,0.0,2,0
3206511,55341,1,37,"""""",0,1,1,2740,0.0,2,0,3,1.20073,1.0,"""SVO""",5,"""SU""",10,1.0,4.0,1.42281,4,0.0,5.58352,0.14635,0.99927,0.0,0.603236,1.014234,1,0.027764,248.162015,197.569236,48.726937,0.403102,0.637398,3,1.112774,0.0,0.0,-45.696381,-4.872694,0.25,52.819673,1370.0,3,1.20073,0.00365,0.001825,3.0,1,0.0,0.0,0.0,2740,0.0,3,0
3479930,60187,0,36,"""""",0,1,1,29,1.0,1,7,8,7.655172,8.0,"""SVO""",2,"""SU""",2,1.0,1.0,1.0,2,0.0,2.578807,0.344828,0.965517,0.0,0.62318,1.0,2,0.325182,64.263881,18.937823,48.522167,0.0,0.61593,1,0.62069,0.0,0.0,-14.162163,-4.852217,0.5,44.261084,29.0,1,0.956897,0.068966,0.0689

In [14]:
# Extract Flight Features
test_df = extract_flight_features(test_df)

In [15]:
# Combine customer and flight features data
test_df = prepare_combined_data(test_df, cust_data)
test_df.head(100)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8
70,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,0.0,0.029412,1.0,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,1.412529,1
71,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",1,3.030303,0.058824,1.259453,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,2.825057,1
72,81,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,15.151515,0.176471,1.881866,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,8.475172,1
73,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,33.333333,0.352941,3.704941,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,16.950344,1
74,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,42.424242,0.441176,4.399117,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,21.18793,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
403,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""99668c849f564e8a849cce15708691…",0,97.14

In [16]:
# Get models if not already loaded
try:
    ranker_models
except NameError:
    ranker_models = joblib.load('data/segment_ranker_models.joblib')

In [17]:
results, test_df = test_segment_based_ranker(ranker_models, test_df)


2025-09-04 15:15:58.306679: Testing segment 0 with 1731254 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.5797

2025-09-04 15:16:18.943906: Testing segment 1 with 38834 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.4767

2025-09-04 15:16:19.605772: Testing segment 2 with 22313 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.0333

2025-09-04 15:16:19.725050: Testing segment 3 with 251514 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.6880

2025-09-04 15:16:24.103888: Testing segment 4 with 38308 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.1039

2025-09-04 15:16:24.393239: Testing segment 5 with 841748 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.4999

2025-09-04 15:16:40.082186: Testing segment 6 with 34924 rows
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.2418

2

In [18]:
test_df.head(100)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match,y_pred
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8,f64
70,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,0.0,0.029412,1.0,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,1.412529,1,-1.795636
71,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",1,3.030303,0.058824,1.259453,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,2.825057,1,-2.318678
72,81,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,15.151515,0.176471,1.881866,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,8.475172,1,-5.992605
73,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,33.333333,0.352941,3.704941,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,16.950344,1,-6.708278
74,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,42.424242,0.441176,4.399117,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,21.18793,1,-8.199756
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
403,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""99668c849f564e8a849cce15708691…",0,97.1428

In [21]:
# Evaluate overall hit rate for segmented prediction
from src.ranking_eval import calculate_hit_rate_at_k

y_true = test_df['selected'].to_numpy()
y_pred = test_df['y_pred'].to_numpy()
group_sizes = (
        test_df
        .group_by('ranker_id', maintain_order=True)
        .len()
        .select('len')
        .to_numpy()
        .flatten()
    )
ranker_ids = test_df['ranker_id'].unique().sort()
hitrate, details = calculate_hit_rate_at_k(y_true, y_pred, group_sizes, ranker_ids, k=3)

In [22]:
hitrate

0.5615348176219801

In [46]:
# Test if full model is better for predicting on segments with low results scores
_ = test_lgb_ranker(ranker_models[99], test_df.filter(pl.col('customer_segment') == 7).drop('y_pred'))

Preparing test data...
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.0000


In [56]:
test_df = test_df.drop('y_pred')

In [92]:
# Except for segment 7, all other segments performed better under the full training model
test_results = test_lgb_ranker(ranker_models[99], test_df, k=3, min_size=10)

Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.5153


In [93]:
# Let's compare segmented prediction vs. general model hit rates, not counting groups smaller than 10 flights
for i in range(len(ranker_models) - 1):
    print(f'Results for segment {i} using segment model:')
    _ = test_lgb_ranker(ranker_models[i], test_df.filter(pl.col('customer_segment') == i), k=3, min_size=10)
    print(f'Results for segment {i} using general model:')
    _ = test_lgb_ranker(ranker_models[99], test_df.filter(pl.col('customer_segment') == i), k=3, min_size=10)

Results for segment 0 using segment model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.5256
Results for segment 0 using general model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.5318
Results for segment 1 using segment model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.3947
Results for segment 1 using general model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.4704
Results for segment 2 using segment model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.0333
Results for segment 2 using general model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.1333
Results for segment 3 using segment model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.6330
Results for segment 3 using general model:
Making predictions...
Calculating Hit Rate @ 3...
Overall Hit Rate @ 3: 0.6344
Results for segment 4 us

In [81]:
test_df = test_df.with_columns(pl.Series('y_pred', test_results[2]))
test_df.head(100)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match,y_pred
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8,f64
70,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,0.0,0.029412,1.0,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,1.412529,1,-1.561822
71,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",1,3.030303,0.058824,1.259453,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,2.825057,1,-1.677042
72,81,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,15.151515,0.176471,1.881866,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,8.475172,1,-5.210053
73,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,33.333333,0.352941,3.704941,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,16.950344,1,-6.604056
74,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,42.424242,0.441176,4.399117,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,21.18793,1,-7.178081
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
403,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""99668c849f564e8a849cce15708691…",0,97.1428

In [82]:
# Rank the predictions for each flight search result by ranker_id
test_df = test_df.with_columns(pl.col('y_pred').rank(method='ordinal', descending=True).over('ranker_id').alias('flight_rank'))

In [89]:
test_df.head(100)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match,y_pred,flight_rank
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8,f64,u32
70,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,0.0,0.029412,1.0,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,1.412529,1,-1.561822,3
71,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",1,3.030303,0.058824,1.259453,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,2.825057,1,-1.677042,4
72,81,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,15.151515,0.176471,1.881866,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,8.475172,1,-5.210053,11
73,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,33.333333,0.352941,3.704941,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,16.950344,1,-6.604056,16
74,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,42.424242,0.441176,4.399117,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,21.18793,1,-7.178081,18
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
403,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""99668c849f564e8a849cce15708691…",0,97.142857,0.97183

In [90]:
test_df = test_df.drop(['y_pred', 'flight_rank'])
test_df.head()

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,selected,price_percentile,price_rank_pct,price_ratio_to_min,route_popularity,route_popularity_log,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_days,leg0_duration_minutes,leg1_duration_minutes,trip_duration_minutes,leg0_departure_hour,leg0_departure_weekday,leg0_arrival_hour,leg0_arrival_weekday,leg1_departure_hour,…,route_loyalty,hub_preference,short_haul_preference,connection_tolerance,preferred_duration_quartile,price_to_duration_sensitivity,avg_price_per_minute,price_per_minute_variance,price_position_preference,premium_economy_preference,consistent_price_tier,preferred_price_tier,baggage_qty_preference,baggage_weight_preference,loyalty_program_utilization,convenience_priority_score,loyalty_vs_price_index,planning_consistency_score,luxury_index,search_intensity_per_route,lead_time_variance,lead_time_skew,carrier_diversity,airport_diversity,cabin_class_range,customer_tier,vip_search_intensity,vip_carrier_diversity,vip_cabin_preference,corp_search_volume,corp_roundtrip_pref,corp_planning_variance,customer_segment,daytime_alignment,weekend_alignment,price_preference_match,carrier_loyalty_match
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,i64,f64,f64,f64,u32,f64,i32,i8,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,i32,i32,i32,i8,i8,i8,i8,i8,…,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64,f64,f64,i8,f64,f64,f64,i64,f64,i32,i64,f64,f64,f64,i8
70,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,0.0,0.029412,1.0,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,1.412529,1
71,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",1,3.030303,0.058824,1.259453,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,2.825057,1
72,81,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,15.151515,0.176471,1.881866,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,8.475172,1
73,81,4600.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,33.333333,0.352941,3.704941,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,16.950344,1
74,81,4600.0,0.0,1.0,4600.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",0,42.424242,0.441176,4.399117,41,3.73767,1,0,"""KHV""",0,0,0,1,0,1,80,0,3,80,0,80,13,1,15,1,0,…,0.971429,0.0,0.0,1.257143,1,0.891595,228.853889,169.39112,48.025974,0.528571,0.626136,2,1.257143,0.0,0.0,-28.041566,-4.802597,0.333333,54.584416,35.0,2,1.971429,0.042857,0.028571,1.0,1,0.0,0.0,0.0,70,0.0,2,0,1.0,0.0,21.18793,1


In [84]:
test_results[1]

[{'search_id': '000316523497473cac3bd9d842e56083',
  'group_size': np.uint32(297),
  'num_actual_selections': 1,
  'hit': False,
  'top_k_scores': [0.1501308247908657,
   0.15809204897971416,
   0.2776039010512166],
  'actual_selections_scores': [-1.2718318620176887]},
 {'search_id': '000f11a0da72447fad06f6ce66b8b4bc',
  'group_size': np.uint32(20),
  'num_actual_selections': 1,
  'hit': True,
  'top_k_scores': [0.08768302778822289,
   0.3531206957271648,
   1.6693302951934568],
  'actual_selections_scores': [1.6693302951934568]},
 {'search_id': '000f1d4460fc4f1595d5eb045b85e2fb',
  'group_size': np.uint32(326),
  'num_actual_selections': 1,
  'hit': False,
  'top_k_scores': [-0.3726256761498053,
   -0.2596676080792344,
   -0.23520481828997822],
  'actual_selections_scores': [-0.9478206160309343]},
 {'search_id': '0015f39518664fbea30587bcd71ef925',
  'group_size': np.uint32(12),
  'num_actual_selections': 1,
  'hit': True,
  'top_k_scores': [-9.134125181218856, -6.708466997101994, -5.0

In [87]:
test_df.filter(pl.col('ranker_id') == '005379e21a624c47a69bc8807a2628bf')['Id', 'ranker_id', 'selected', 'y_pred', 'flight_rank'].sort('flight_rank')

Id,ranker_id,selected,y_pred,flight_rank
i64,str,i64,f64,u32
5130750,"""005379e21a624c47a69bc8807a2628…",0,-0.079849,1
5130752,"""005379e21a624c47a69bc8807a2628…",1,-0.131826,2
5130702,"""005379e21a624c47a69bc8807a2628…",0,-0.608402,3
5130700,"""005379e21a624c47a69bc8807a2628…",0,-0.631898,4
5131040,"""005379e21a624c47a69bc8807a2628…",0,-0.690141,5
…,…,…,…,…
5131367,"""005379e21a624c47a69bc8807a2628…",0,-11.106838,964
5131309,"""005379e21a624c47a69bc8807a2628…",0,-11.183963,965
5131305,"""005379e21a624c47a69bc8807a2628…",0,-11.214876,966
